# License Status Prediction - Multiclass Classification
<hr style="border:1px solid gray"> </hr>

## Contents.

    1. [x] Problem Description
    2. [x] Data Exploration
    3. [ ] Feature Selection
    4. [ ] KNN Algorithm
    5. [ ] Naive Bias
    5. [ ] Logistic Regression
    6. [ ] Decision Tree
    
    
--------------------------------------------------------------------------------------------------------------------------------


### Problem Description

Based on the following license dataset (real world problem - which holds various information related the business license), I'am going to predict license status for the given business. Lets start with delving deep into the data.
________________________________________________________________________________________________________________________________

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# will make your plot outputs appear and be stored within the notebook.
%matplotlib inline
import os
import category_encoders as ce

In [2]:
df = pd.read_csv("../dataset/License_dataset.csv")
df.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION,LICENSE STATUS
0,35342-20020816,1256593,32811,1,CARMEN CAHUE,CLAUDIA'S BRIDAL SHOP,2625 S CENTRAL PARK AVE 1,CHICAGO,IL,60623.0,...,2002-08-16T00:00:00,2003-08-15T00:00:00,2002-08-21T00:00:00,2006-04-11T00:00:00,NaN,25.0,41.843613,-87.714618,"{'latitude': '41.843612879431845', 'longitude'...",AAI
1,1358463-20051116,1639294,262311,29,"ISLA TROPICAL, INC.",ISLA TROPICAL,2825 W MONTROSE AVE,CHICAGO,IL,60618,...,2005-11-16T00:00:00,2006-11-15T00:00:00,2006-04-05T00:00:00,2006-06-12T00:00:00,2006-06-15T00:00:00,60.0,41.961132,-87.699626,"{'latitude': '41.96113244107215', 'longitude':...",AAC
2,1980233-20090722,1980233,345008,1,DJS REMODELING,"DJS REMODELING, INC.",1605 CLAVEY RD 1,HIGHLAND,IL,60035,...,2009-07-22T00:00:00,2011-07-15T00:00:00,2009-07-22T00:00:00,2009-07-22T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI
3,1476582-20040211,1476582,273121,1,ALL-BRY CONSTRUCTION CO.,ALL-BRY CONSTRUCTION CO.,8 NORTH TRAIL,LEMONT,IL,60439,...,2004-02-11T00:00:00,2005-02-15T00:00:00,2004-02-10T00:00:00,2004-02-11T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI
4,1141408-20080516,1896750,213785,1,MCDONOUGH MECHANICAL SERVICE,MCDONOUGH MECHANICAL SERVICE,4081 JOSEPH DR,WAUKEGAN,IL,60087,...,2008-05-16T00:00:00,2010-05-15T00:00:00,2008-06-04T00:00:00,2008-06-05T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI


In [3]:
# change the columns name
new_col_name = [col.replace(" ", "_").lower() for col in df.columns]
df.columns   = new_col_name

In [4]:
df.head(545)

,id,license_id,account_number,site_number,legal_name,doing_business_as_name,address,city,state,zip_code,...,license_term_start_date,license_term_expiration_date,license_approved_for_issuance,date_issued,license_status_change_date,ssa,latitude,longitude,location,license_status
0,35342-20020816,1256593,32811,1,CARMEN CAHUE,CLAUDIA'S BRIDAL SHOP,2625 S CENTRAL PARK AVE 1,CHICAGO,IL,60623.0,...,2002-08-16T00:00:00,2003-08-15T00:00:00,2002-08-21T00:00:00,2006-04-11T00:00:00,NaN,25.0,41.843613,-87.714618,"{'latitude': '41.843612879431845', 'longitude'...",AAI
1,1358463-20051116,1639294,262311,29,"ISLA TROPICAL, INC.",ISLA TROPICAL,2825 W MONTROSE AVE,CHICAGO,IL,60618,...,2005-11-16T00:00:00,2006-11-15T00:00:00,2006-04-05T00:00:00,2006-06-12T00:00:00,2006-06-15T00:00:00,60.0,41.961132,-87.699626,"{'latitude': '41.96113244107215', 'longitude':...",AAC
2,1980233-20090722,1980233,345008,1,DJS REMODELING,"DJS REMODELING, INC.",1605 CLAVEY RD 1,HIGHLAND,IL,60035,...,2009-07-22T00:00:00,2011-07-15T00:00:00,2009-07-22T00:00:00,2009-07-22T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI
3,1476582-20040211,1476582,273121,1,ALL-BRY CONSTRUCTION CO.,ALL-BRY CONSTRUCTION CO.,8 NORTH TRAIL,LEMONT,IL,60439,...,2004-02-11T00:00:00,2005-02-15T00:00:00,2004-02-10T00:00:00,2004-02-11T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI
4,1141408-20080516,1896750,213785,1,MCDONOUGH MECHANICAL SERVICE,MCDONOUGH MECHANICAL SERVICE,4081 JOSEPH DR,WAUKEGAN,IL,60087,...,2008-05-16T00:00:00,2010-05-15T00:00:00,2008-06-04T00:00:00,2008-06-05T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1646940-20051114,1646940,297075,1,CARMELO MARTINEZ & CELLESTE REYES DE OJEDA,SUPER MERCADO EL GURERRERITO,6001 S KEDZIE AVE,CHICAGO,IL,60629,...,2005-11-14T00:00:00,2006-11-15T00:00:00,2005-11-10T00:00:00,2005-11-14T00:00:00,2006-05-12T00:00:00,59.0,41.784339,-87.703144,"{'latitude': '41.784338681497566', 'longitude'...",AAC
541,1223633-20030216,1326107,230177,1,"Witech Company, Inc.","Witech Company, Inc.",24242 S VOLBRECHT RD,CRETE,IL,60417,...,2003-02-16T00:00:00,2004-02-15T00:00:00,2003-02-14T00:00:00,2003-02-21T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI
542,1228290-20030216,1326864,245090,1,"Roe Construction, Inc.","Roe Construction, Inc.",11264 SOUTHWEST HWY,PALOS HILLS,IL,60465,...,2003-02-16T00:00:00,2004-02-15T00:00:00,2003-05-07T00:00:00,2003-05-08T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI
543,1248775-20040216,1453346,28655,2,A T M I,A T M I,960 RIDGEWAY AVE,AURORA,IL,60506,...,2004-02-16T00:00:00,2005-02-15T00:00:00,2003-12-22T00:00:00,2004-01-21T00:00:00,NaN,NaN,NaN,NaN,NaN,AAI


## Variable Description

**Dependent Variable**  <br>
1. AAI - License status is issued <br>
2. AAC - License status is cancelled <br>
3. REV - License status is revoked <br>
4. REA - License status is revoked and appealed <br>
5. INQ - License status is in enquiry <br>

**Independent Variable(to predict):** <br>
* Timeline of the application status <br>
* Type of business <br>
* Location details of the business <br>
* Payment details <br>
_______________________________________________________________________________________________________________________________

In [5]:
df.shape

(85895, 32)

## Data Cleansing


![title](../images/data-cleasing.png)

1. Missing Values - lot of missing data for particular variables
2. Unique Data - id (no meaning)
3. Data leakage - The AAI alone doesn't have license status change value (as the license are never revoked/cancelled) 

In [6]:
df.isnull().sum()

id                                       0
license_id                               0
account_number                           0
site_number                              0
legal_name                               0
doing_business_as_name                   1
address                                  0
city                                     0
state                                    0
zip_code                                31
ward                                 49701
precinct                             56701
ward_precinct                        49700
police_district                      54012
license_code                             0
license_description                      0
license_number                           1
application_type                         0
application_created_date             64660
application_requirements_complete      214
payment_date                          1289
conditional_approval                     0
license_term_start_date                228
license_ter

In [7]:
# Drop columns which are not relevent for the prediction / too many missing values
drop_col_list = ["id","license_id","ssa","location","application_created_date","account_number","address"]
df = df.drop(drop_col_list, axis=1)

In [8]:
# just mapping whenever a True of False value exists -> 1, 0
df["license_status_change"] = np.where(df.license_status_change_date.isnull(),1,0)

In [9]:
pd.crosstab(df.license_status_change, df.license_status)

license_status,AAC,AAI,INQ,REA,REV
license_status_change,,,,,
0,30200,0,2,3,290
1,0,55400,0,0,0


## Data Transformation
1. Timeline creation
2. Encoding
3. New Feature

In [10]:
strings_cols =  ["application_requirements_complete", 
                 "payment_date", 
                 "license_term_start_date",
                 "license_term_expiration_date",
                 "license_approved_for_issuance",
                 "date_issued"] 

def str_to_date_format(columns: list):
    for col in columns:
        df[col] = pd.DatetimeIndex(df[col])

str_to_date_format(strings_cols)


In [11]:
type(df['license_status'])

pandas.core.series.Series

In [12]:
type(df[['license_status']])

pandas.core.frame.DataFrame

In [13]:
# Find #days btw different application status date
df["completion_to_start"]   = (df.license_term_start_date - df.application_requirements_complete).dt.days
df["start_to_expiry"]       = (df.license_term_expiration_date - df.license_term_start_date).dt.days
df["approval_to_issuance"]  = (df.date_issued - df.license_approved_for_issuance).dt.days
df["completion_to_payment"] = (df.payment_date - df.application_requirements_complete).dt.days

df["presence_of_enquiry_details"] = np.where(df.ward.isnull() | df.ward_precinct.isnull() | df.police_district | df.precinct , 0 ,1 )

# creating one hot encondig 
df["target"] = df[['license_status']].apply(lambda col:pd.Categorical(col).codes)